In [1]:
import csv
def loadCsv(filename):
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [2]:
import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [3]:
import math
import numpy as np
def mean(numbers):
    numbers = np.array(numbers)
    
    f1 = numbers[:,0].astype(float)
    f2 = numbers[:,1].astype(float)
    #print(f1)
    #print(f2)
    xx = np.mean(f1)
    yy = np.mean(f2)
    
    cov = np.cov(f1,f2)
    mean = (xx,yy)
    m = list(mean)
    
    return m,cov

In [58]:
def update_mean_cov(data,r):
    K = 3
    wght = []
    mean_ = []
    aa = 0
    N = len(data)
    print(N)
    data = np.array(data)
    count = 0
    
    #update weights for liklihood calculation
    for i in range(K):
        aa = 0
        for j in range(N):
            aa += r[i][j]
        wght.append(aa)
    #print(wght)
    #update mean 
    for i in range(2):
        for j in range(N):
            count += data[j][i] * r[i][j]
        total = count/len(data)
        mean_.append(total)
    print(mean_)    
    
    return mean_ ,wght

In [59]:
def summarize(dataset):
    #for x in range(3):
    #print(dataset,'dssd',len(dataset))
    #summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    summaries = []
    m = mean(dataset)
    #cv = covm(dataset)
    summaries.append(m)
    #summaries.append(cv)
    
    #del summaries[-1]
    #print(summaries)
    return summaries

In [60]:
def seperateBylabels(dataset,df):
    datasets = {}
    by_class = df.groupby('label')

    for groups, data in by_class:
        datasets[groups] = data
        
    #print(datasets)
    return datasets

In [61]:
def summarizeByClass(dataset,df):
    separated = seperateBylabels(dataset,df)
    #print(separated)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    #print(summaries)
    return summaries

In [62]:
def divide_mean_cov(summaries):
    mean_K = []
    cov_K = []
    for classValue, classSummaries in summaries.items():
        for i in range(len(classSummaries)):
            mean, cov = classSummaries[i]
            #print(mean,cov,'\n')
            mean_K.append(mean)
            cov_K.append(cov)
    return mean_K,cov_K

In [63]:
import math
import operator
import numpy as np
def Normal(x, mean, cov):
    #pp = np.array(x) - np.array(mean)
    
    #determinant of covariance matrix
    det_cov = np.linalg.det(cov)
    pp = list(map(operator.sub,x,mean)) 
    #print(x,mean)
    #print('covariance',cov)
    #print('x-mean',pp)
    
    
    #inverse of the covariance matrix
    inv_cov = np.linalg.inv(cov)
    
    pt = np.array(pp).reshape(2,1)
    parta = np.dot(pp,inv_cov)
    partb = np.dot(parta,pt)
    part1 = math.exp(-1/2 * partb)
    #print(part1)
    
    part2 = 2 * math.pi * math.sqrt(det_cov)
    #print('part2',part2)
    
    gauss = part1/part2
    
    #print('transpose',pt)
    #inv_cov = np.linalg.det(cov)
    #print(gauss)
    #print(parta)
    #print
    
    return gauss

In [64]:
def maximizeLLH(data, mean, cov,alpha):
    liklihood = []
    
    data = np.array(data)
    for i in range(len(data)):
        temp = 0
        for j in range(3):
            mean_n = mean[j]
            cov_n = cov[j]
        #print(mean_n, cov_n)
            result = alpha[j] * Normal(data[i], mean_n, cov_n)
            temp = temp + result
        temp1 = np.log(temp)
        print(temp1)
    return temp1

In [65]:
def Estep(mean,cov,data,alpha,r):
    K = 3
    print('Estep')
    s = np.zeros(len(data))
    data = np.array(data)
    for i in range(len(data)):
        temp = np.zeros(K)
        for j in range(3):
            temp[j] = alpha[j] * Normal(data[i], mean[j], cov[j])
            s[i] += temp[j]
        for k in range(K):
            r[k][i] = temp[k]/s[i]
    return r,s

In [66]:
def Mstep(mean,cov,data,alpha,r,s):
    print("Enter M step.")
    N = len(data)
    data = np.array(data)
    K = 3
    for k in range(K):
        # Calculate alpha[k]
        alpha[k] = np.sum(r[k]) / N
        #print(alpha)
        #calculate mean
        total = np.zeros(2)
        print(total)
        for i in range(N):
            #print(data[i])
            #print(r[k][i])
            total += r[k][i] * data[i]
            print(total)
        mean[k] = total / np.sum(r[k])
        print(mean[k])

In [67]:
def execute(mean,cov,data,alpha):
    K = 3
    r = np.zeros([K,len(data)])
    #estimate(mean,cov,data)
    #lld = maximizeLLH(data, mean, cov,alpha)
    r,s = Estep(mean,cov,data,alpha,r)
    mean_up,wght_up = update_mean_cov(data,r)
    #print(r)
    #Mstep(mean,cov,data,alpha,r,s)
    

In [68]:
from sklearn.cluster import KMeans
def doKmeans(X, nclust):
    model = KMeans(nclust)
    model.fit(X)
    return (model.labels_, model.cluster_centers_)

In [69]:
import numpy as np
import pandas as pd

def main():
    alphas = [31/150, 85/150, 34/150]
    K = 3
    alpha = np.array(alphas)
    filename = r'C:\Users\Govind\Downloads\02-Govind\LS_Group02\task_boundary.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet,testSet = splitDataset(dataset, splitRatio)
    print('Size of trainingSet is ',len(trainingSet),'and testing size is ',len(testSet))
    
    
    trainingSet = np.array(trainingSet)
    
    df = pd.DataFrame(trainingSet, columns = ['x', 'y','class'])
    clust_labels, cent = doKmeans(trainingSet, 3)
    
    df['label'] = clust_labels
    
    train = []
    for x in range (len(trainingSet)):
        a = trainingSet[x][-1]
        train.append(a)
    
    trainData = df.drop(['class'],axis = 1)
    #print(trainData)
    
    
    parameters = summarizeByClass(trainData,df)
    #print(parameters)
    mean,cov = divide_mean_cov(parameters)
    #print(mean,cov,'\n')
    execute(mean,cov,trainData,alpha)
    
    
main()

Size of trainingSet is  1005 and testing size is  495
Estep
1005
[-1.6416529896384606, -0.986439314127832]
